In [26]:
import pandas as pd
import numpy as np
df = pd.read_csv('model_data.csv').drop(columns = ['Unnamed: 0','f_highrate','f_lowrate'])

In [27]:
df.columns

Index(['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status',
       'S_create2SA_Create', 'S_Create2Start_Time', 'SA_Create2Start_Time',
       'U_create2now', 'U_approve2now', 'prev_CW x SA_rate', 'type_RN',
       'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no',
       'areaName_dfw', 'areaName_austin', 'areaName_san', 'net_pay', 'target',
       'sa_create', 'Start_Time', 'CW_in_a_month', 'count_prev_SA',
       'count_prev_CW'],
      dtype='object')

# Data Prepration

In [28]:
# standardlize, dont standardlize dummy! 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.drop(['id', 'user_id', 'shift_id', 'status', 'target', 'sa_create', 'Start_Time', 'type_RN', 
                    'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no', 'areaName_dfw', 
                    'areaName_austin', 'areaName_san', 'CW_in_a_month'], axis=1))

scaled_features = scaler.transform(df.drop(['id', 'user_id', 'shift_id', 'status', 'target', 'sa_create',
                                            'Start_Time', 'type_RN', 'type_LVN+LPN', 'segmentName_d', 
                                            'areaName_houston', 'areaName_no', 'areaName_dfw', 
                                            'areaName_austin', 'areaName_san', 'CW_in_a_month'], axis=1))

# scaled features
X = pd.DataFrame(scaled_features, columns = ['prev_CW/SA_rate', 'prev_CW x SA_rate', 'S_create2SA_Create', 
                                             'S_Create2Start_Time', 'SA_Create2Start_Time', 'U_create2now', 
                                             'U_approve2now', 'net_pay', 'count_prev_SA', 'count_prev_CW'])
# concat with dummy
df = pd.concat([df[['id', 'user_id', 'shift_id', 'status', 'target', 'sa_create',
                    'Start_Time', 'type_RN', 'type_LVN+LPN', 'segmentName_d', 
                    'areaName_houston', 'areaName_no', 'areaName_dfw', 
                    'areaName_austin', 'areaName_san', 'CW_in_a_month']], X], axis = 1)

# drop nas
df.dropna(inplace = True)

### Slice df by the end of this week, for predcition output

In [29]:
end_of_week = '2021-4-12'

# convert to datetime for conditonal selection
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
# sort by start time -> for slicing
df = df.sort_values(by = 'Start_Time') 
# record as realdata
realdata = df[df['Start_Time'].apply(lambda x: x > pd.to_datetime(end_of_week))]
# record predction output rows, don't include it in tran test validation
realdata_len = realdata.shape[0]
# only keep status = confirmed
realdata = realdata[realdata['status'] == 'confirmed']

###  <font color = green> Validation set: 1000 recently records

In [30]:
# slice, dont include realdata
validation = df[-1000-realdata_len : -realdata_len]

y_valid = validation['target']
x_valid = validation.drop(['id','user_id', 'shift_id', 'status', 'sa_create', 'Start_Time', 'target'], axis = 1)

y_valid.value_counts()

0    906
1     94
Name: target, dtype: int64

### Train test: main dataset - validation set

In [31]:
df = df[:-1000-realdata_len] # slice 

In [32]:
# # make a dataset that num of tar = num of non tar, use it for train test
# import random
# df_tar = df[df['target']==1].reset_index(drop = True)
# df_nontar = df[df['target']==0].reset_index(drop = True)

# number_of_tar = df_tar.shape[0]
# random_indices = random.sample(range(len(df_nontar)), int(number_of_tar))
# df_nontar = df_nontar[df_nontar.index.isin(random_indices)]

# # concat
# df = pd.concat([df_tar, df_nontar]).reset_index(drop = True)

In [33]:
X = df.drop(['id','user_id', 'shift_id', 'status', 'target', 'sa_create', 'Start_Time'], axis = 1)
y = df['target']

# set test, train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [34]:
df['target'].value_counts()

0    59611
1     4695
Name: target, dtype: int64

# Logistic Regression 1  

In [35]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# assign less punlishment for classifying 0 as 1 -> find more 1's
# weights = {0:1, 1:10}
# class_weight = 'balanced': automatically adjust weights inversely proportional to class frequencies in the input data
logit = LogisticRegression(solver = 'lbfgs', max_iter=100000, class_weight = 'balanced')
logit.fit(X_train,y_train)

LogisticRegression(class_weight='balanced', max_iter=100000)

### Train Test result

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = logit.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print('\n')
print(classification_report(y_test, y_pred))

[[11716  6167]
 [  460   949]]


              precision    recall  f1-score   support

           0       0.96      0.66      0.78     17883
           1       0.13      0.67      0.22      1409

    accuracy                           0.66     19292
   macro avg       0.55      0.66      0.50     19292
weighted avg       0.90      0.66      0.74     19292



In [37]:
# from cf_matrix import make_confusion_matrix
# labels = ['True Neg','False Pos','False Neg','True Pos']
# categories = ['Zero', 'One']
# make_confusion_matrix(confusion_matrix(y_test, y_pred), 
#                       group_names=labels,
#                       categories=categories, 
#                       cmap='Blues')

In [38]:
# logit summary
import statsmodels.api as sm
smlogit = sm.Logit(y_train,X_train).fit()
smlogit.summary()

Optimization terminated successfully.
         Current function value: 0.240966
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 target   No. Observations:                45014
Model:                          Logit   Df Residuals:                    44996
Method:                           MLE   Df Model:                           17
Date:                Sun, 11 Apr 2021   Pseudo R-squ.:                 0.07792
Time:                        22:20:35   Log-Likelihood:                -10847.
converged:                       True   LL-Null:                       -11763.
Covariance Type:            nonrobust   LLR p-value:                     0.000
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
type_RN                 -3.4955      0.172    -20.336      0.000      -3.832      -3.159
type_LVN+LPN            -1.4177      0.078    -18.172      0.000      -1.571      -1.265
segmentName_d            0.6139      0.097      6.298      0.000       0.423       0.805
areaName_houston        -2.9502      0.092    -32.216      0.000      -3.130      -2.771
areaName_no             -2.9175      0.102    -28.726      0.000      -3.117      -2.718
areaName_dfw            -2.9283      0.107    -27.481      0.000      -3.137      -2.719
areaName_austin         -2.7952      0.125    -22.309      0.000      -3.041      -2.550
areaName_san            -2.6507      0.130    -20.385      0.000      -2.906      -2.396
CW_in_a_month            0.2471      0.046      5.428      0.000       0.158       0.336
prev_CW/SA_rate          0.1190      0.015      8.201      0.000       0.091       0.147
prev_CW x SA_rate        0.1165   1.23e+06   9.46e-08      1.000   -2.41e+06    2.41e+06
S_create2SA_Create      -0.1174   1.87e+06  -6.28e-08      1.000   -3.66e+06    3.66e+06
S_Create2Start_Time     -0.2616   1.39e+06  -1.89e-07      1.000   -2.72e+06    2.72e+06
SA_Create2Start_Time     0.0474      0.056      0.851      0.395      -0.062       0.157
U_create2now            -0.1915      0.057     -3.380      0.001      -0.303      -0.080
U_approve2now            0.1289      0.032      4.006      0.000       0.066       0.192
net_pay                  0.9974      0.041     24.469      0.000       0.917       1.077
count_prev_SA           -0.2234      0.036     -6.143      0.000      -0.295      -0.152
count_prev_CW            0.1109      0.029      3.886      0.000       0.055       0.167
========================================================================================
"""

### Overfitting? No

In [39]:
y_pred = logit.predict(X_train)

print(confusion_matrix(y_train, y_pred))
print('\n')
print(classification_report(y_train, y_pred))

[[27571 14157]
 [ 1114  2172]]


              precision    recall  f1-score   support

           0       0.96      0.66      0.78     41728
           1       0.13      0.66      0.22      3286

    accuracy                           0.66     45014
   macro avg       0.55      0.66      0.50     45014
weighted avg       0.90      0.66      0.74     45014



In [40]:
# test threshold
limiter = .65

y_prob = list(logit.predict_proba(X_train)[:,1])
y_pred = []
count =0
for prob in y_prob:
    if prob >= limiter:
        y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)

print(confusion_matrix(y_train, y_pred))
print('\n')
print(classification_report(y_train, y_pred))

[[36824  4904]
 [ 2126  1160]]


              precision    recall  f1-score   support

           0       0.95      0.88      0.91     41728
           1       0.19      0.35      0.25      3286

    accuracy                           0.84     45014
   macro avg       0.57      0.62      0.58     45014
weighted avg       0.89      0.84      0.86     45014



### <font color = green> Validation result

In [41]:
# test threshold
limiter = .65

y_prob = list(logit.predict_proba(x_valid)[:,1])
y_pred = []
count =0
for prob in y_prob:
    if prob >= limiter:
        y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)

print(confusion_matrix(y_valid, y_pred))
print('\n')
print(classification_report(y_valid, y_pred))

[[702 204]
 [ 42  52]]


              precision    recall  f1-score   support

           0       0.94      0.77      0.85       906
           1       0.20      0.55      0.30        94

    accuracy                           0.75      1000
   macro avg       0.57      0.66      0.57      1000
weighted avg       0.87      0.75      0.80      1000



# Fit real data in this model

In [44]:
# set input
real_X = realdata.drop(['id','user_id', 'shift_id', 'status', 'target', 'sa_create', 'Start_Time'], axis = 1)

In [45]:
# concat predicted prob with data
realdata['prob'] = list(logit.predict_proba(real_X)[:,1])

In [46]:
# record when this prediction is ran
from datetime import date
time = str(date.today().year) + '-' + str(date.today().month) + '-' + str(date.today().day)

realdata[['id', 'Start_Time', 'prob']].to_csv('pred_{}_Silver_Bullet.csv'.format(time), index = False)

In [47]:
# to make the prediction doesn't include today
realdata['Start_Time']

41768   2021-04-12 05:00:00
46540   2021-04-12 05:00:00
55352   2021-04-12 05:00:00
66259   2021-04-12 05:00:00
65792   2021-04-12 05:00:00
                ...        
66399   2021-05-08 06:30:00
64668   2021-05-08 18:00:00
66400   2021-05-11 06:30:00
66401   2021-05-15 06:30:00
66402   2021-05-22 06:30:00
Name: Start_Time, Length: 770, dtype: datetime64[ns]